#### 1. Retrieve data from excel files

In [1]:
import pandas as pd
import numpy as np


# 輸入完整檔名 (記得file type也要加入: .xlsx / .csv)
main_excel = "FY23 Audit 必修實體及線上課程完成率.xlsx"
link_excel = "FY23_E-learning_link.xlsx"
saba_csv = "Learner Completions Report.csv"

# 注意sheet_name，未來請Learning Team統一
df = pd.read_excel("import_data/"+ main_excel, header=None)
df_course1 = pd.read_excel("import_data/" + link_excel, header=3, sheet_name='Experienced Hire必修線上專業課程') # course link 1
df_course2 = pd.read_excel("import_data/" + link_excel, header=2, sheet_name='各職級必修線上專業課程') # course link 2
df_course3 = pd.read_excel("import_data/" + link_excel, sheet_name='GCOE專業課程') # course link 3
df_saba = pd.read_csv("import_data/" + saba_csv, skiprows=16)

#### 2. 處理attend_all分頁

In [2]:
df_empID = df.iloc[11:, 1:2].dropna()
df_empID.columns = df_empID.iloc[0]
df_empID = df_empID.drop(df_empID.index[0])

# 對一下原始資料前五個員工編號
df_empID.head() 

14,員工編號
15,90028901
16,91105392
17,90036542
18,90698052
19,90035372


In [3]:
"""
attend_all tab (EXCEL大表--右下角)
"""

# Main
df_attend = df.iloc[:, 25:]                                                   # Excel Z欄後
df_attend.drop(df_attend.index[0], inplace=True)                              # delete useless rows
df_attend.drop(df_attend.index[1:8], inplace=True)                            # delete useless rows
df_attend.drop(df_attend.index[3:6], inplace=True)                            # delete useless rows
df_attend.iloc[1:3] = df_attend.iloc[1:3].ffill(axis=1)                       # 處理excel置中- fill course name & course number(to the right)
df_attend = df_attend.replace(r'\n',' ', regex=True)                          # replace new line 
df_attend.iloc[0,:] = df_attend.replace(np.nan, '否').bfill().iloc[0,:]       # "前一年度Plan是否有？" 空白補上 "否"
df_attend.columns = df_attend.iloc[0,:].apply(str) +'!'+ df_attend.iloc[1,:].apply(str) +'!'+ df_attend.iloc[2,:].apply(str)+'!'+ df_attend.iloc[3,:].apply(str)
df_attend.drop(df_attend.index[0:4], inplace=True)                            # remove first 3 rows
df_attend = df_attend.rename(columns = {df_attend.columns[0]:'員工編號'})   #rename column
df_attend['員工編號']  = df_empID['員工編號']                                # merge employee id
df_attend = df_attend[df_attend['員工編號'].notna()].fillna(0.0)
df_attend = df_attend.set_index('員工編號').stack().reset_index()               # transform table
df_attend[['前一年度Plan是否有','course_number', '課程名稱', 'hours']] = df_attend.level_1.str.split('!', expand=True)
df_attend = df_attend.drop(columns='level_1')
df_attend = df_attend[df_attend['course_number'].str.contains("---") == False]     # future warning!!!
df_attend_toPiv = df_attend.copy()

df_plan = df_attend[['course_number', '前一年度Plan是否有']].drop_duplicates(subset="course_number")

df_attend_toPiv = df_attend_toPiv.pivot(index=['員工編號','course_number', '課程名稱'],
                                        columns='hours',
                                        values=0).reset_index(drop=False)

df_attend_toPiv = df_attend_toPiv.rename(columns = {'實際時數':'實上', '必修時數(應上)':'應上'})
res_df_attend = df_attend_toPiv.merge(df_plan, how='left', on='course_number')



need_vals = ['取消(年休假/事假/其他)','取消(因公)','取消(預計離職)','待完成','待補課']   # 列入完成率計算
noNeed_vals= ['取消(婚假/產假)','取消(病假)','取消(喪假)','取消(公假/溫書假)']          # 不列入完成率計算

res_df_attend.loc[res_df_attend['實上'].isin(noNeed_vals), '應上'] = 0
res_df_attend.loc[res_df_attend['實上'].isin(need_vals + noNeed_vals), '實上'] = 0


res_df_attend = (res_df_attend.drop(['應上'], axis=1).join(res_df_attend['應上'].apply(pd.to_numeric, errors='coerce')))
res_df_attend['實上'] = np.where(res_df_attend['實上'].map(np.isreal) == False, res_df_attend['應上'], res_df_attend['實上'])  #處理梯次+奇怪的值...轉成[應上]的值
res_df_attend = (res_df_attend.drop(['實上'], axis=1).join(res_df_attend['實上'].apply(pd.to_numeric, errors='coerce')))

res_df_attend = res_df_attend.rename(columns={'前一年度Plan是否有':'年度'})
res_df_attend.loc[res_df_attend['年度'] == '是', '年度'] = '前年度'
res_df_attend.loc[res_df_attend['年度'] == '否', '年度'] = '今年度'
res_df_attend['備註'] = np.where(res_df_attend['應上'] - res_df_attend['實上'] != 0, '待完成', '已完成')
res_df_attend = res_df_attend.drop(columns='課程名稱')
res_df_attend['course_number'] = res_df_attend['course_number'].str.strip()

res_df_attend = res_df_attend[res_df_attend["應上"] != 0.0]
res_df_attend.head()

,員工編號,course_number,年度,應上,實上,備註
8,90006761,GL-AUD-0000302731,前年度,0.25,0.0,待完成
9,90006761,GL-AUD-0000320407,今年度,0.37,0.0,待完成
10,90006761,GL-AUD-0000334150,今年度,0.33,0.0,待完成
11,90006761,GL-AUD-0000339674,今年度,0.25,0.0,待完成
12,90006761,GL-AUD-0000339721,今年度,1.00,0.0,待完成


#### 3. 處理Employee分頁

In [4]:
"""
EMPLOYEE tab (EXCEL大表--左下角)
"""
df_emp = df.iloc[14:, :16] # row, column

df_emp.columns = df_emp.iloc[0,:]
df_emp = df_emp.drop(df_emp.index[0])
df_emp = df_emp.drop(df_emp.columns[[7, 8, 13, 14]], axis=1) 

# FY23未升遷
df_emp['FY23未升遷'] = df_emp['FY23未升遷'].fillna('有升遷')
df_emp = df_emp.rename(columns = {df_emp.columns[-5]:'是否升遷'})

# GCoE學員
df_emp['GCoE學員'] = df_emp['GCoE學員'].fillna('否')
df_emp.loc[df_emp['GCoE學員'] == 'Yes', 'GCoE學員'] = '是'

# 前一年度取得資格者
df_emp['前一年度取得資格者'] = df_emp['前一年度取得資格者'].fillna('否')
df_emp.loc[df_emp['前一年度取得資格者'] == 'Yes', '前一年度取得資格者'] = '是'

# A20B對應Audit職級要排的課
df_emp['A20B對應Audit職級要排的課'] = df_emp['A20B對應Audit職級要排的課'].fillna('')

# 年度中到職(12/1以後)
df_emp['年度中到職(12/1以後)'] = df_emp['年度中到職(12/1以後)'].fillna('否')
# df_emp.loc[df_emp['年度中到職(12/1以後)'] != '否', '年度中到職(12/1以後)'] = '是'

df_emp.head()

14,組別,員工編號,職稱,中文姓名,英文姓名,e-Mail,分機,是否升遷,GCoE學員,前一年度取得資格者,A20B對應Audit職級要排的課,年度中到職(12/1以後)
15,A001,90028901,資深會計師,李東峰,Titan Lee,titanlee@deloitte.com.tw,3912,有升遷,否,否,,否
16,A001,91105392,資深經理,王柔方,Sydney J. Wang,sywang@deloitte.com.tw,5059,有升遷,否,否,,否
17,A003,90036542,資深會計師,洪玉美,May Hung,mayhung@deloitte.com.tw,5531,有升遷,否,否,,否
18,A003,90698052,資深會計師,王曉琪,Sherrie Dobrow,sherridobrow@deloitte.com.tw,5941,有升遷,是,是,,否
19,A003,90035372,資深會計師,顏曉芳,Karen Yen,karenyen@deloitte.com.tw,8503,有升遷,是,是,,否


#### 4. 處理Course分頁

In [5]:
"""
course tab (EXCEL大表--右上角)
"""
df_course = df.iloc[:11, 25:]
df_course = df_course.dropna(axis=1, how="all").T #remove all nan columns
df_course.columns = df_course.iloc[0]
df_course = df_course.rename(columns = {df_course.columns[-2]:'課程編碼'})

df_course = df_course.drop(df_course.index[0])
df_course['實體/Virtual/線上'] = df_course['實體/Virtual/線上'].ffill()
df_course['前一年度Plan是否有？'] = df_course['前一年度Plan是否有？'].fillna('否')
df_course['ISA/US'] = df_course['ISA/US'].fillna('-') 
df_course['GCoE必修'] = df_course['GCoE必修'].fillna('')
df_course['課程負責人'] = df_course['課程負責人'].fillna('')
df_course["課程編碼"] = df_course["課程編碼"].astype("string")

df_course = df_course[df_course['課程編碼'].str.contains("---") == False]     # future warning!!!


df_course['結算一'] = pd.to_datetime(df_course['結算一']).dt.date
df_course['結算二'] = df_course['結算二'].replace('---', '')
df_course['結算二'] = pd.to_datetime(df_course['結算二']).dt.date


df_course = df_course[df_course['課程編碼'].notna()]
df_course['課程編碼'] = df_course['課程編碼'].str.strip()

# COURSE 1+2+3
df_course1 = df_course1[['Course code', '課程連結']].astype("string")
df_course2 = df_course2[['Course code', '課程連結']].astype("string")
df_course3 = df_course3[['Course code', '課程連結']].astype("string")
df_course_merged = pd.concat([df_course1, df_course2, df_course3], ignore_index=True)       #跟SQL的UNION一樣
df_course_merged = df_course_merged.replace(r'\n',' ', regex=True)                          # replace new line 

# 將大表LEFT JOIN課程連結
df_merged = df_course.merge(df_course_merged, how='left', left_on='課程編碼', right_on='Course code')  #LEFT JOIN
df_merged = df_merged.drop(columns='Course code')
df_merged = df_merged.drop_duplicates()
df_merged.head()

,實體/Virtual/線上,前一年度Plan是否有？,ISA/US,職級,GCoE必修,課程負責人,應完成日期,結算一,結算二,課程編碼,課程名稱,課程連結
0,實體/Virtual,否,-,Staff I,,Viktoria#2689,七月新生：7/1、7/4-7/5\n九月新生：9/1-9/2、9/5-9/6\n10、11月...,2022-12-31,2023-05-31,TWSTFI2303,新生基礎課程,<NA>
1,實體/Virtual,否,-,Staff I,,Viktoria#2689,第一梯：11/14-11/18\n第二梯：11/21-11/25\n第三梯：11/28-12/2,2022-12-31,2023-05-31,TWSTFI2305,查核實作系列_審計員I (Jump In),<NA>
2,實體/Virtual,否,-,Staff I~P,,Viktoria#2689,C1：8/25-8/26上午\nC2：9/19-9/20上午\nC3：9/20下午-9/21...,2022-12-31,2023-05-31,TWOT2303,G250 The Deloitte Way for VSAs\n(Staff I & Exp...,<NA>
3,實體/Virtual,否,-,Staff II,,Coco#5543,第一梯：9/12-9/15\n第二梯：9/27-9/30\n第三梯：11/22-11/25,2022-12-31,2023-05-31,TWSTFII2302,查核實作系列_審計員II（Raise the Bar),<NA>
4,實體/Virtual,否,-,Staff II,,Coco#5543,第一梯：9/16(2班)\n第二梯：9/26(2班)\n第三梯：11/21(2班)\n北區：...,2022-12-31,2023-05-31,TWSTFII2303,Staff II課程B,<NA>


#### 5. 處理SABA分頁

In [6]:
df_saba['Completion Date'] =  df_saba['Completion Date'].str[-4:] +'/'+ df_saba['Completion Date'].str[3:5] +'/'+ df_saba['Completion Date'].str[:2]    #處理'Completion Date'格式
df_saba['Person Custom5/Person Personnel No'] = df_saba['Person Custom5/Person Personnel No'].str[2:]   # 刪除'Person Custom5/Person Personnel No'最前面的"tw"
df_saba['Course Course ID'] = df_saba['Course Course ID'].replace(r'\n',' ', regex=True).str.strip()
df_saba.head()

,Course Title,Course Course ID,Course Vendor,Course CPE Hours,Course Field of Study,Course Learning Hours,Class Duration,Class ID,Class Delivery Name,Person First Name,...,Person Custom3/Person Service Line,Person Organisation Name,Person Custom4/Person Cost Center Number,Person Custom5/Person Personnel No,Person Job Type Name,Person Business Card Title/Person Global Level,Person Job Type Internal Description,Completion Status,Completed Courses (Transcript) Score,Completion Date
0,新進同仁溝通技巧,TW-MD-COMM-NH,NaN,NaN,NaN,0.5,8,1511314,Digital Learning,LENA W.,...,NaN,HC,850010012,92121042,TW-Senior,senior,Senior,Successful,0.0,2021/06/22
1,新進同仁溝通技巧,TW-MD-COMM-NH,NaN,NaN,NaN,0.5,8,1511314,Digital Learning,EVA Y.,...,NaN,TP,850010007,92382322,TW-Staff,staff,Staff,Successful,0.0,2021/06/22
2,新進同仁溝通技巧,TW-MD-COMM-NH,NaN,NaN,NaN,0.5,8,1511314,Digital Learning,GENNIE H.,...,NaN,TP,850010004,92200792,tw-asst. manager,senior,asst. manager,Successful,0.0,2021/06/20
3,新進同仁溝通技巧,TW-MD-COMM-NH,NaN,NaN,NaN,0.5,8,1511314,Digital Learning,LOUIS C.,...,NaN,HC,850010012,91936921,tw-asst. manager,senior,asst. manager,Successful,0.0,2021/06/23
4,新進同仁溝通技巧,TW-MD-COMM-NH,NaN,NaN,NaN,0.5,8,1511314,Digital Learning,RITA C.,...,NaN,HC,850010012,92179922,tw-asst. manager,senior,asst. manager,Successful,0.0,2021/06/28


#### 6. 取資料區間

In [64]:
period = df.iloc[1, 0].split("：",1)[1]
dict_period = {'data_period': period}
df_period = pd.DataFrame(data=dict_period, index=[0])
df_period

,data_period
0,2022/06/01-2022/10/16


#### 7. Save To File

In [94]:
with pd.ExcelWriter(r'output/result.xlsx', engine='xlsxwriter') as writer:
    res_df_attend.to_excel(writer, sheet_name='QRM059_attend_all', index=False)
    df_emp.to_excel(writer, sheet_name='QRM059_employee', index=False, encoding='utf-8')
    df_merged.to_excel(writer, sheet_name='QRM059_course', index=False)
    df_saba.to_excel(writer, sheet_name='QRM059_LearnerCompletion', index=False)
    df_period.to_excel(writer, sheet_name='QRM059_data_period', index=False)

---------------------------------------------------

#### **安裝Pyinstaller** - py to exe

- 正常來說只要: ! pip install <package_name>


- 由於事務所電腦會擋權限，因此要加入信任參數來安裝:

    ! pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org <package_name>

In [ ]:
! pip install --trusted-host pypi.org --trusted-host files.pythonhosted.org pyinstaller

#### 將python檔案變成exe

- 步驟一: 將所有code放在同一個cell裡
- 步驟二: 將ipynb檔，另存成py檔
- 步驟三: 執行下面的code，將app.py更改為上一步驟的檔名 (pyinstaller.exe --onefile --windowed app.py)

! pyinstaller.exe --onefile --windowed app.py